In [1]:
from fileinput import filename
import os
import random

import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from utils import settings as settings
import pandas as pd
from evaluation.metrics import accuracy
from evaluation.conf_matrix import create_conf_matrix
from loader.Preprocessor import Preprocessor
from models.JensModel import JensModel
from models.RainbowModel import RainbowModel
from models.ResNetModel import ResNetModel
from utils.filter_activities import filter_activities
from utils.folder_operations import new_saved_experiment_folder
from utils.DataConfig import Sonar22CategoriesConfig, OpportunityConfig, SonarConfig, LabPoseConfig
from tensorflow.keras.layers import (Dense)
from utils.Recording import Recording
import matplotlib.pyplot as plt

2022-05-25 11:28:44.826363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 11:28:44.826407: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import utils.DataConfig
import importlib
importlib.reload(utils.DataConfig)
from utils.DataConfig import Sonar22CategoriesConfig, OpportunityConfig, SonarConfig, LabPoseConfig

# Init
# TODO: refactor, find out why confusion matrix sometimes shows different results than accuracy
# TODO: - make train and test datasets evenly distributed
#       - make 
""" 
Number of recordings per person
{'connie.csv': 6, 'alex.csv': 38, 'trapp.csv': 9, 'anja.csv': 13, 'aileen.csv': 52, 'florian.csv': 16, 'brueggemann.csv': 36, 'oli.csv': 20, 'rauche.csv': 9, 'b2.csv': 6, 'yvan.csv': 8, 'christine.csv': 7, 'mathias.csv': 2, 'kathi.csv': 17}
"""

data_config = LabPoseConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
data_config = SonarConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
settings.init(data_config)
random.seed(1678978086101)

k_fold_splits = 3
numEpochsBeforeTL = 10
numEpochsForTL = 3
minimumRecordingsPerLeftOutPerson = 5
# Load dataset
recordings = settings.DATA_CONFIG.load_dataset()#limit=75


/dhc/groups/bp2021ba1/data/lab_data/1651152887499
took: 717.575 - 0.135 - 9.784 - 15.312
1.0
/dhc/groups/bp2021ba1/data/lab_data/1652085113670
took: 364.612 - 0.110 - 4.682 - 7.162
1.0
/dhc/groups/bp2021ba1/data/lab_data/1651223183731
took: 1176.808 - 0.101 - 13.914 - 30.870
1.0
/dhc/groups/bp2021ba1/data/lab_data/1651565920786
took: 769.237 - 0.103 - 12.749 - 19.519
1.0
/dhc/groups/bp2021ba1/data/lab_data/1652708794017
took: 14175.600 - 0.104 - 18.095 - 15.965
1.0
/dhc/groups/bp2021ba1/data/lab_data/1651567699911
took: 970.994 - 0.123 - 14.994 - 27.976
1.0
/dhc/groups/bp2021ba1/data/lab_data/1652431073226
took: 1055.750 - 0.143 - 12.356 - 21.776
1.0
/dhc/groups/bp2021ba1/data/lab_data/1651149014709
took: 158.703 - 0.088 - 2.944 - 2.748
1.0
/dhc/groups/bp2021ba1/data/lab_data/1651674149525
took: 365.205 - 0.108 - 5.833 - 7.788
16.0
/dhc/groups/bp2021ba1/data/lab_data/1652432054525
took: 1197.372 - 0.103 - 17.259 - 26.207
1.0
/dhc/groups/bp2021ba1/data/lab_data/1652426607642
took: 29187

In [75]:
import pose_sequence_loader
import importlib
importlib.reload(pose_sequence_loader)

from pose_sequence_loader import *
from multiprocessing import Pool
import tqdm

recordings[0].pose_frame = pd.DataFrame()

def process_recording(recording_with_index):
    i = recording_with_index[0]
    recording = recording_with_index[1]
    pose_frame = get_poseframe(recording)
    
    print(f"Pose Frame added to Recording {i+1}/{len(recordings)}  ")
    return pose_frame

# pose_frames = []
# for i, k in list(enumerate(recordings)):
#     pose_frames.append(process_recording((i,k)))

pool = Pool()
#for _ in tqdm.tqdm(
pose_frames = pool.map(process_recording, list(enumerate(recordings)), 1)
#total=len(recordings)
#):
#   pass
pool.close()
pool.join()

for i, pose_frame in enumerate(pose_frames):
    recordings[i].pose_frame = pose_frame

Pose Frame added to Recording 8/36  
Pose Frame added to Recording 2/36  
Pose Frame added to Recording 9/36  
Recording 10 failed to get sampled up. Returning empty pose frame (!)
Pose Frame added to Recording 11/36  
Pose Frame added to Recording 1/36  
Pose Frame added to Recording 5/36  
Pose Frame added to Recording 4/36  
Pose Frame added to Recording 6/36  
Pose Frame added to Recording 7/36  
Pose Frame added to Recording 3/36  
Pose Frame added to Recording 12/36  
Pose Frame added to Recording 15/36  
Pose Frame added to Recording 14/36  
Pose Frame added to Recording 19/36  
Pose Frame added to Recording 22/36  
Pose Frame added to Recording 10/36  
Pose Frame added to Recording 13/36  
Pose Frame added to Recording 16/36  
Pose Frame added to Recording 20/36  
Pose Frame added to Recording 23/36  
Pose Frame added to Recording 17/36  
Pose Frame added to Recording 26/36  
Pose Frame added to Recording 24/36  
Pose Frame added to Recording 28/36  
Pose Frame added to Recordi

In [ ]:
recording = recordings[1]

activities = recording.activities
first_activity = activities[0]
for i in range(len(activities)):
    if first_activity != activities[i]:
        break

frame_first_time = recording.time_frame[0]
frame_time_second_activity = recording.time_frame[i]
frame_diff = round((frame_time_second_activity - frame_first_time) / 1000)

print(i, frame_diff)

In [40]:
for i, pose_frame in enumerate(pose_frames):
    recordings[i].pose_frame = pose_frame



In [77]:

#for recording in recordings:
#    print(recording.sensor_frame.shape[0], recording.pose_frame.shape[0], recording.time_frame.shape[0], recording.activities.shape[0])
print(recordings[0].pose_frame)

# with open("/dhc/groups/bp2021ba1/alex/UnicornML/src/pose_frame_0.csv", "w+") as file:
#     recordings[0].pose_frame.to_csv(file)

       LEFT_SHOULDER_X  LEFT_SHOULDER_Y  RIGHT_SHOULDER_X  RIGHT_SHOULDER_Y  \
0            -1.000000        -1.000000         -1.000000         -1.000000   
1            -1.000000        -1.000000         -1.000000         -1.000000   
2            -1.000000        -1.000000         -1.000000         -1.000000   
3            -1.000000        -1.000000         -1.000000         -1.000000   
4            -1.000000        -1.000000         -1.000000         -1.000000   
...                ...              ...               ...               ...   
22245         0.517216         0.308133          0.401618          0.311061   
22246         0.516831         0.307694          0.402074          0.311085   
22247         0.515897         0.307002          0.402552          0.310392   
22248         0.515018         0.306350          0.403002          0.309740   
22249         0.514084         0.305658          0.403480          0.309048   

       LEFT_ELBOW_X  LEFT_ELBOW_Y  RIGHT_ELBOW_X  R